In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import json

In [ ]:
plt.rcParams['figure.figsize'] = 6, 4.5
plt.rcParams['figure.subplot.bottom'] = 0.15
plt.rcParams['figure.subplot.hspace'] = 0.05

format_graphique = '.eps'

def format_simplifie(x, pos):
    """
    Format simplifié pour l'axe Y des graphiques logarithmiques
    
    Le résultat affiche les décimales uniquement pour les valeurs plus
    petites que 1
    """
    if x >1:                                      
        return '%1.0f' % x     
    else:                      
        return '%g' % x     


In [ ]:
with open('traductions.json', 'r') as f:
    traduction = json.load(f)
    
langue = 'en'
def trad(texte):
    if langue == 'fr':
        return texte
    else:
        return traduction[langue][texte]

In [ ]:
# Paramètres du modèle SEIR
# Données de l'article Wang et al


σ = 1/5.2 # Inverse de la période d'incubation
γ = 1/18  # Inverse du temps de recouvrement

N = 11e6  # Taille de la population

# Définition du modèle SEIR, Eq. TODO de l'article

def SEIR(x, t, R0):
    S, E, I, R = x         # x est la variable vecteur contenant les quatre variables du modèle
    β = R0(t)*γ
    return [
            - β*I*S/N,     # Dérivée de S
            β*I*S/N - σ*E, # Dérivée de E
            σ*E - γ*I,     # Dérivée de I
            γ*I,           # Dérivée de R
           ]

# Routine pour résoudre le modèle

def solution_SEIR(I0, E0, R0, temps):
    solution = odeint(SEIR, (N-I0-E0, I0, E0, 0), temps, args=(R0,))
    return map(np.array, zip(*solution))

In [ ]:
temps = np.linspace(0, 119, 300) # Valeur de la variable temps, en unités de jour

def R0_constant(R0):
    def f(t):
        return R0
    return f


fraction_de_cas = 0.1

unite = 1_000_000

I0 = 100 / fraction_de_cas
E0 = I0*20

S, E, I, R = solution_SEIR(E0, I0, R0_constant(4), temps)

f = plt.figure()
ax1 = f.add_subplot(211)
ax2 = f.add_subplot(212, sharex=ax1)


for i, scale in enumerate(['linear', 'log']):

    ax_right = f.add_subplot(2, 1, i+1, frameon=False)
    plt.tick_params(labelcolor=(0, 0, 0, 0), top=False, bottom=False, left=False, right=False)

    plt.sca(ax1 if i==0 else ax2)
    ax = plt.gca()

    plt.plot(temps, S/unite, label='S', ls='-.')
    plt.plot(temps, E/unite, label='E', ls='--')
    plt.plot(temps, I/unite, label='I')
    plt.plot(temps, R/unite, label='R', ls=':')

    plt.ylabel(trad('Population\n[Millions de personnes]'))

    plt.yscale(scale)
    plt.grid()
    plt.legend()


    if i==1:
        ax.set_xlabel(trad('Temps [Jours]'))


    if scale == 'log':
        ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_simplifie))
    else:
        pass

    ylo, yhi = plt.ylim()
    plt.ylim(max(2e-3, ylo), yhi)
    ax_right.set_ylabel(trad('Linéaire' if scale=='linear' else 'Logarithmique'))
    ax_right.yaxis.set_label_position('right')

ax2.set_xlim(temps.min(), temps.max())
plt.savefig('modele_SEIR_'+langue+format_graphique)

In [ ]:
temps = np.linspace(0, 240, 300) # Valeur de la variable temps, en unités de jour

def R0_constant(R0):
    def f(t):
        return R0
    return f

fraction_de_cas = 0.1
fraction_usi = 0.07
n_lits = 8000

unite = 1_000_000

I0 = 100 / fraction_de_cas
E0 = I0*20


f = plt.figure()
ax0 = f.add_subplot(111, frameon=False)
plt.tick_params(labelcolor=(0, 0, 0, 0), top=False, bottom=False, left=False, right=False)

ax1 = f.add_subplot(211)
ax2 = f.add_subplot(212, sharex=ax1)

ax0.set_ylabel(trad('Population\n[Millions de personnes]'))
l_styles = ['-', '--', ':', '-.']

for i, scale in enumerate(['linear', 'log']):

    ax_right = f.add_subplot(2, 1, i+1, frameon=False)
    plt.tick_params(labelcolor=(0, 0, 0, 0), top=False, bottom=False, left=False, right=False)

    plt.sca(ax1 if i==0 else ax2)
    ax = plt.gca()
    
    for j, R0_value in enumerate([1.9, 3.1, 4]):
        S, E, I, R = solution_SEIR(E0, I0, R0_constant(R0_value), temps)
        plt.plot(temps, I/unite, label=f'{trad("I pour")} R0={R0_value}', ls=l_styles[j])

    plt.yscale(scale)
    plt.grid()

    plt.axhline(n_lits / fraction_usi / fraction_de_cas / unite, label=trad('Saturation lits USI'),
                color='cyan')
    plt.legend()

    if i==1:
        ax.set_xlabel(trad('Temps [Jours]'))

    if scale == 'log':
        ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_simplifie))
    else:
        pass

    ylo, yhi = plt.ylim()
    plt.ylim(max(2e-3, ylo), yhi)
    ax_right.set_ylabel(trad('Linéaire' if scale=='linear' else 'Logarithmique'))
    ax_right.yaxis.set_label_position('right')

ax1.set_xlim(0, 240)

plt.savefig('scenarios_SEIR_'+langue+format_graphique)

In [ ]:
temps = np.linspace(0, 240, 300) # Valeur de la variable temps, en unités de jour

def R0_constant(R0):
    def f(t):
        return R0
    return f

fraction_de_cas = 0.1
fraction_usi = 0.07
n_lits = 8000

unite = 1_000_000

I0 = 100 / fraction_de_cas
E0 = I0*20


f = plt.figure()
ax0 = f.add_subplot(111, frameon=False)
plt.tick_params(labelcolor=(0, 0, 0, 0), top=False, bottom=False, left=False, right=False)

ax1 = f.add_subplot(211)
ax2 = f.add_subplot(212, sharex=ax1)

ax0.set_ylabel('Population\n[Millions de personnes]')
l_styles = ['-', '--', ':', '-.']

for i, scale in enumerate(['linear', 'log']):

    ax_right = f.add_subplot(2, 1, i+1, frameon=False)
    plt.tick_params(labelcolor=(0, 0, 0, 0), top=False, bottom=False, left=False, right=False)

    plt.sca(ax1 if i==0 else ax2)
    ax = plt.gca()

    for j, R0_value in enumerate([1, 1.9, 3.1, 4]):
        S, E, I, R = solution_SEIR(E0, I0, R0_constant(R0_value), temps)
        plt.plot(temps, R/unite, label=f'R pour R0={R0_value}', ls=l_styles[j])

    plt.yscale(scale)
    plt.grid()

    plt.axhline(n_lits / fraction_usi / fraction_de_cas / unite, label='Saturation lits USI',
                color='cyan')
    plt.legend()

    if i==1:
        ax.set_xlabel('Temps [Jours]')

    if scale == 'log':
        ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_simplifie))
    else:
        pass

    ylo, yhi = plt.ylim()
    plt.ylim(max(2e-3, ylo), yhi)
    ax_right.set_ylabel('Linéaire' if scale=='linear' else 'Logarithmique')
    ax_right.yaxis.set_label_position('right')

ax1.set_xlim(0, 240)


In [ ]:
temps = np.linspace(0, 1080, 2000) # Valeur de la variable temps, en unités de jour

def R0_constant(R0):
    def f(t):
        return R0
    return f


fraction_de_cas = 0.1
n_lits = 2000

unite = 1_000_000

I0 = 1 / fraction_de_cas
E0 = I0*20


f = plt.figure()

l_styles = ['-', '--', ':', '-.']

for j, R0_value in enumerate([1.3, 1.9, 3.1, 4]):
    S, E, I, R = solution_SEIR(E0, I0, R0_constant(R0_value), temps)
    plt.plot(temps/30, I/unite, label=f'I pour R0={R0_value}', ls=l_styles[j])
    
plt.axhline(n_lits / fraction_de_cas / 0.05 / unite)

plt.yscale(scale)
plt.grid()
plt.legend()

plt.xlabel('Temps [Mois]')

#ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(format_simplifie))

ylo, yhi = plt.ylim()
plt.ylim(max(2e-4, ylo), yhi)

